In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
pd.set_option('display.max_columns', 500)
data=pd.read_csv('bank-data.csv',sep=';')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


# Prep Data

In [15]:
#convert YES/NO to No=0 and Yes = 1
data['y']=le.fit_transform(data['y'])
data['loan']=le.fit_transform(data['loan'])
data['default']=le.fit_transform(data['default'])
data['housing']=le.fit_transform(data['housing'])


In [16]:
#convert category data to dummies columns
data = pd.concat([data,pd.get_dummies(data['job'],prefix='job')],axis=1).drop(columns='job')
data = pd.concat([data,pd.get_dummies(data['marital'],prefix='marital')],axis=1).drop(columns='marital')
data = pd.concat([data,pd.get_dummies(data['education'],prefix='education')],axis=1).drop(columns='education')
data = pd.concat([data,pd.get_dummies(data['contact'],prefix='contact')],axis=1).drop(columns='contact')
data = pd.concat([data,pd.get_dummies(data['month'],prefix='month')],axis=1).drop(columns='month')
data = pd.concat([data,pd.get_dummies(data['poutcome'],prefix='poutcome')],axis=1).drop(columns='poutcome')

In [17]:
data.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,0,2143,1,0,5,261,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,44,0,29,1,0,5,151,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,33,0,2,1,1,5,76,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,47,0,1506,1,0,5,92,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,33,0,1,0,0,5,198,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


# Train & Test Data

In [18]:
data_train = data.sample(frac=0.7)
data_test = data.drop(data_train.index)

#check 
pd.crosstab(data_train['y'],columns='COUNT')
pd.crosstab(data_test['y'],columns='COUNT')

data_train_y = data_train['y']
data_train_x = data_train.drop(columns='y')
data_test_y = data_test['y']
data_test_x = data_test.drop(columns='y')

# Model

ต้องลง Library Graphvi ก่อน

เปิด Anaconda > Environment > Open Terminal

pip install graphviz
or
conda install graphviz
แนะนำใช้ตัวนี้

In [19]:
import graphviz

In [37]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(min_samples_leaf=30,max_depth=5)

clf = clf.fit(data_train_x,data_train_y)

import graphviz
dot_data = tree.export_graphviz(clf,\
                                feature_names=data_train_x.columns,\
                                class_names=['0','1'],\
                                filled=True,\
                               rotate=True)

graph=graphviz.Source(dot_data)
graph.render('dtree_render3.pdf')

data_feature = pd.DataFrame({'Feature':data_train_x.columns,'Score':clf.feature_importances_})

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tpdf', '-O', 'dtree_render3.pdf'], make sure the Graphviz executables are on your systems' PATH

In [34]:
data_feature.sort_values(by='Score',ascending=False).head()

,Feature,Score
6,duration,0.565508
46,poutcome_success,0.297783
0,age,0.053610
39,month_mar,0.042857
31,contact_unknown,0.011146


# Predictive & Accuracy

In [35]:
res = clf.predict(data_test_x)
pd.crosstab(data_test_y , res)

col_0,0,1
y,,
0,11624,343
1,969,627


In [36]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print("Accuracy : \t %.3f"%accuracy_score(data_test_y,res))
print(classification_report(data_test_y,res))

Accuracy : 	 0.903
              precision    recall  f1-score   support

           0       0.92      0.97      0.95     11967
           1       0.65      0.39      0.49      1596

    accuracy                           0.90     13563
   macro avg       0.78      0.68      0.72     13563
weighted avg       0.89      0.90      0.89     13563

